Author
Mette Dybdahl Mortensen

In [1]:
import numpy as np
from numpy import pi
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


About the project

This project is a demonstration of how the band gap arises in the nearly free electron model. In the program I have used a square lattice and introduced a periodic potential. The size of the band gap can be controlled by changing the strength of the potential. The potential can also be turned off completely and we are left with the normal dispersion relation E = hbar/(2m)*(kx^2+ky^2).
In the program I have chosen to set hbar=1 and m=1.
It is also possible to plot the Fermi level. Depending on where this Fermi level is placed the material will either be a metal, a semi-conductor or an insulator. If one of the bands are completely full the material will either be a semi-conductor or an insulator depending on the size of the gap. If one of the bands are not completely full the electrons will be able to conduct and we have a metal.
If the atoms on the lattice each have one valence electron the lower band will be half full. For a semi-conductor the Fermi level will lie (approximately) in the middle of the band gap.
It is also possible to arrange the potential and Fermi level such that the Fermi level cuts two bands. This will result in a semi-metal.


How to control the program

* The strength of the potential can be  easiest regulated by changing the value of V0 (e.g. V0=3.0).
* The height of the Fermi level can be regulated by changing the value of Ef (e.g. Ef=5.0)
* More bands can be added to the plot by changing the xrange in line 71 (xrange(0,2) gives the two lowest bands xrange(0,3) the three lowest and so on)


In [2]:

kpoints=100

a = 1.0 #lattice constant
G = 2*pi/a
n = np.arange(-1,2)
nsize = np.size(n)
nsize2 = nsize**2
V0 = 0.0 #strength of the perioic potential

kxs = np.linspace(-pi/a, pi/a, kpoints)
kys = np.linspace(-pi/a, pi/a, kpoints)

coeffs = np.zeros(nsize2) #Fourier coefficients of the potential 
coeffs[1::2]=0.5 * V0
coeffs[0] = 0.5 * V0

#Create the Hamiltonian
H = np.zeros((nsize2,nsize2))

for i in range(1,nsize2):
    offdiag = coeffs[i] * np.ones(nsize2 - i)
    H = H + (np.diag(offdiag,k=i) + np.diag(offdiag,k=-i))
    
    
def Energy(q,p):
    for i in range(0,nsize):
        for j in range(0,nsize):
            H[nsize*i+j,nsize*i+j] = 0.5*((q + n[i]*G)**2 + (p + n[j]*G)**2) + coeffs[0]
    return np.real(np.sort((np.linalg.eigvals(H)))) #Find eigenvalues of the Hamiltonian


#Arrange the values in energy bands
bands = np.array([Energy(-pi/a,-pi/a)])

for kx in kxs:
    for ky in kys:
        bands = np.append(bands, np.array([Energy(kx,ky)]), axis=0)
        
bands = np.delete(bands,0,0)


#Create a GL View widget to display data
app = QtGui.QApplication([])
plot = gl.GLViewWidget()
plot.show()
plot.setWindowTitle('Band gap + Fermi level')
plot.setCameraPosition(distance=50)

#Add a grid
g = gl.GLGridItem()
g.setDepthValue(10)
plot.addItem(g)

#Plot the bands
#Change the values in xrange to add more bands to the plot
for n in range(0,2):
    z = np.reshape(bands[:,n],(100,100))
    bandn = gl.GLSurfacePlotItem(x=kxs, y=kys, z=z, shader='normalColor')
    bandn.scale(3,3,1)
    plot.addItem(bandn)

#Add the Fermi level to the plot
Ef = 5.0 * np.ones((100,100))
fermi = gl.GLSurfacePlotItem(x=kxs, y=kys, z=Ef, shader='shaded', 
                             color=(0.0,0.25,0.5,0.0))
fermi.scale(3,3,1)
plot.addItem(fermi)


#Start Qt event loop unless running in interactive mode
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()